# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.
2. Append all this datasets and generate a unique dataframe. This newdataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**
3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.
4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

In [1]:
import pandas as pd
import datetime as date
import os
from pathlib import Path


# Answer 1:

In [14]:
# Primero se realizó una revisión de la carpeta conteniendo los datos. Se observa que hay datos desde el 2001 hasta el 2022 de las tasas de interés de los
# bancos. Además, existen distintas formas de los formatos. En particular, hay un formato traspuesto desde el 2002 hasta agosto de 2010 y luego un formato
# donde los bancos son las columnas a partir de septiembre de 2010. El caso del 2001 (solo hay una base de dicho año) tiene el formato de los últimos años.

# Como la data presentada está a nivel mensual, pero no contiene información de todos los meses de los años, se procede a hacer un bucle for para cada año, mes y
# día. Sin embargo, para evitar que salga error y deje de correr el bucle, se utiliza el comando 'try' y 'except' para que cuando en la base de datos no tenga la
# combinación requerida de año, mes y día, pase al siguiente paso del bucle.

# Finalmente, a partir del bucle mencionado se importará cada uno de los excel que contienen la información mensual de las tasas de interés de los bancos y se generará un DataFrame (df) por cada mes identificándolo con un número generado a partir de un contador. Este contador nos servirá para: identificar el df donde empieza y termina el cambio de formato de los excel.

contador = 0
for anio in range(2001, 2023):
    for mes in range(1,13):
        for dia in range(26,32):
            try:
                df = pd.read_excel(r"../../_data\sbs\B_RawData\bancos\table_clean_"+str(dia)+"_"+str(mes)+"_"+str(anio)+".xlsx")
                df["id_df"] = contador                          #Almacenamos el número de orden del df en una variable
                df["date_info"] = date.date(anio, mes, dia)     #Almacenamos la fecha del excel en una variable
                locals()[f'df{contador}'] = df                  #Generamos un nuevo df con el 'id' del contador
                contador+=1                                     #Hacemos que el contador crezca solo si se genera una base de datos
                if anio == 2010 and mes == 8 and dia == 31:     #Es la fecha en la que empieza cambia de formato
                    limite = contador
                elif anio == 2001 and mes == 3 and dia == 31:   #En el primer archivo no hay problema de formato
                    inicio = contador
                else:
                    continue
            except:
                continue

print(f'Se exportaron satisfactoriamente ' + str(contador) + ' archivos excel')
print(f'El formato es distinto entre el df' + str(inicio) + ' y el df' + str(limite))

Se exportaron satisfactoriamente 241 archivos excel
El formato es distinto entre el df1 y el df97


# Answer 2:

In [15]:
# Antes de poder juntar todas las bases debemos trabajar los df para el periodo donde se tiene un formato distinto. Para ello, se generará un bucle con la ayuda
# de los objetos 'inicio' y 'limite' generados en la pregunta anterior. El objetivo de este bucle es modificar el formato de las tablas entre el 2002 hasta
# agosto de 2010. Nuevamente, como no hay df para todos los números entre 'inicio' y 'límite' se utiliza el comando 'try' y 'except' para que funcione el bucle

for data in range(inicio,limite):
    alterna = locals()[f'df{data}']                #En esta línea generamos un df "comodín" denominado alterna que usaremos para cambiar el formato de los df.
    fecha = alterna.iloc[ -1 ,-1]                  #Generamos un objeto denominado fecha, que almacene la fecha de cada df
    id = alterna.iloc[-1,-2]                       #Generamos un objeto denominado id que almacena el # de contador asociado a la tabla
    alterna = alterna.rename(columns={ 'Unnamed: 0':''})
    alterna = alterna.iloc[ 1:-1 , 0:-2].T         #Transponemos el df y eliminamos la fila y id de fecha para solo quedarnos con las tasas de interés
    alterna.columns = alterna.iloc[0]              #Modificamos los index de columnas para que sean los bancos (primera fila de los df)
    alterna = alterna.iloc[1:]                     #Dropeamos la primera fila que contiene la info de los bancos
    alterna.columns = alterna.columns.str.lower()  #Modificamos los nombres de las variables para que estén en minúsculas
    alterna["Tasa Anual (%)"] = alterna.iloc[:,0]  #Generamos una variable denominada "Tasa Anual (%)" con la info de los tipos de crédito. Esto porque no nos permite modificar un NaN
    alterna = alterna.iloc[: , 1:]                 #Eliminamos la columna "NaN"
    alterna = alterna[['Tasa Anual (%)'] + [col for col in alterna.columns if col != 'Tasa Anual (%)']] #Ahora reordenamos las columnas, para que Tasa Anual sea la primera
    alterna = alterna.reset_index().iloc[: , 1:]
    alterna["date_info"] = fecha                   #Generamos la variable fecha utilizando el objeto fecha
    alterna["id_df"] = id                          #Generamos la variable id_df utilizando el objeto id
    locals()[f'df{data}'] = alterna                #Actualizamos el df con el df comodín que se generó en el bucle


In [16]:
# Loop para modificar las minúsculas/mayúsculas de cada variable
for data in range(0,contador):
    alterna = locals()[f'df{data}']
    alterna.columns = alterna.columns.str.lower()

In [17]:
df_final = df0
for data in range(1, contador):
    df_final = pd.concat([df_final , locals()[f'df{data}']])

# Answer 3:

In [18]:
# Generamos la variable año
cont = 0
df_final["year"] = ""
for obs in df_final['date_info']:
    df_final["year"].iloc[cont] = obs.strftime("%Y")
    cont+=1

df_final.loc[df_final['tasa anual (%)'] == "HIPOTECARIO", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'
df_final.loc[df_final['tasa anual (%)'] == "Hipotecarios", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'
df_final.loc[df_final['tasa anual (%)'] == "PRÉSTAMOS HIPOTECARIOS", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'

df_final.loc[df_final['tasa anual (%)'] == "Consumo", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de Crédito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito cerradas5", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito abiertas4", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de Crdito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'


condition_1 = df_final['tasa anual (%)'] == "Préstamos hipotecarios para vivienda"
condition_2 = df_final['tasa anual (%)'] == "Consumo - Tarjeta de crédito"


df_filtrada = df_final[condition_1 | condition_2 ]


C:\Users\canun\AppData\Local\Temp\ipykernel_6900\1435163596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["year"].iloc[cont] = obs.strftime("%Y")


In [29]:
melt_df_filtrada = df_filtrada.melt(id_vars=['year', 'tasa anual (%)'], value_vars=[ 'bbva', 'comercio', 'crédito', 'pichincha', 'bif', 'scotiabank', 'citibank', 'interbank', 'mibanco', 'gnb', 'falabella', 'santander', 'ripley', 'alfin', 'icbc', 'bank of china', 'trabajo', 'banco de comercio', 'banco de credito', 'banco pichincha', 'banbif', 'b santander central', 'standard chartered', 'banco sudamericano', 'wiese sudameris', 'bank boston', 'bnp paribas el', 'citibank del peru', 'scotiabank peru', 'hsbc(*)', 'hsbc', 'banco falabella', 'santander peru', 'banco ripley', 'alfin banco', 'deutsche bank peru', 'continental', 'financiero', 'azteca', 'deutsche', 'azteca  *', 'cencosud', 'banco gnb', 'financiero  *', 'cat', 'bci'], value_name='tasa')

dict_bancos = {'continental': 'bbva', 'banco de comercio': 'comercio', 'banco de credito': 'crédito', 'banco pichincha': 'pichincha', 'financiero': 'pichincha', 'financiero  *': 'pichincha', 'banbif': 'bif', 'scotiabank peru': 'scotiabank', 'banco sudamericano': 'scotiabank', 'wiese sudameris': 'scotiabank', 'citibank del peru': 'citibank', 'banco gnb': 'gnb', 'hsbc(*)': 'gnb', 'hsbc': 'gnb', 'banco falabella': 'falabella', 'b santander central': 'santander', 'santander peru': 'santander', 'banco ripley': 'ripley', 'alfin banco': 'alfin', 'azteca': 'alfin', 'azteca  *': 'alfin', 'deutsche bank peru': 'deutsche', 'trabajo': 'trabajo', 'standard chartered': 'standard chartered', 'bank boston': 'bank boston', 'bnp paribas el': 'bnp paribas el', 'cencosud': 'cencosud', 'cat': 'cat', 'bci': 'bci', 'bbva':'bbva', 'comercio':'comercio', 'crédito':'crédito', 'pichincha':'pichincha', 'bif':'bif', 'scotiabank':'scotiabank', 'citibank':'citibank', 'gnb':'gnb' , 'falabella':'falabella', 'santander':'santander' , 'ripley':'ripley', 'alfin':'alfin', 'deutsche' :'deutsche', 'interbank' : 'interbank', 'icbc' : 'icbc', 'bank of china' : 'bank of china', 'mibanco':'mibanco'}

melt_df_filtrada['variable'] = melt_df_filtrada['variable'].map(dict_bancos)


In [30]:
melt_df_filtrada['tasa'] = pd.to_numeric(melt_df_filtrada['tasa'], errors='coerce')

year_list = list(set(melt_df_filtrada['year']))
tasa_list = list(set(melt_df_filtrada['tasa anual (%)']))

contador3 = 0
for year in year_list:
    for rate in tasa_list:
        contador3+=1
        condition_3 = melt_df_filtrada['year'] == year
        condition_4 = melt_df_filtrada['tasa anual (%)'] == rate
        df_alterna = melt_df_filtrada[condition_3 & condition_4]
        if rate == "Consumo - Tarjeta de crédito":
            df_alterna = df_alterna.sort_values(by=['tasa anual (%)' , 'tasa'], ascending=[True, False])
        else:
            df_alterna = df_alterna.sort_values(by=['tasa anual (%)' , 'tasa'], ascending=[False, False])
        df_alterna = df_alterna.iloc[0:5, :]
        if contador3 == 1 :
            df_top5 = df_alterna
        else:
            df_top5 = pd.concat([df_top5 , df_alterna])

df_top5 = df_top5.rename(columns={ 'tasa anual (%)':'rate_concept', 'variable':'banks', 'tasa':'rate_value'})
df_top5 = df_top5.sort_values(by=['year', 'rate_concept' , 'rate_value'], ascending=[True, False, False])

# Answer 4:

In [31]:
# First we create a new folder using relative paths.

os.mkdir(r"../../_output\sbs\group_9")
relative_path = Path(r"../../_output\sbs\group_9")
bank_list = list(set(df_top5['banks']))

contador4 =0
for bank in bank_list:
    bank_list[contador4] = bank_list[contador4].replace(" ", "")
    bank_list[contador4] = bank_list[contador4].replace("*", "_")
    contador4+=1

for bank in bank_list:
    condition = df_top5['banks'] == bank
    bank_alterna = df_top5[condition]
    bank_alterna.to_csv(f"{relative_path}/{bank}.csv", index= False )
